# Data Collection

Data is collected from mainly 2 sources : 
- https://www.malayalachalachithram.com/
- https://en.wikipedia.org/wiki/List_of_Malayalam_films_of_[year]
    
Libraries used :
- BeautifulSoup : a library that makes it easy to scrape information from web pages. https://pypi.org/project/beautifulsoup4/
- html5lib :  a pure-python library for parsing HTML. https://pypi.org/project/html5lib/

In [1]:
#importing all libraries required

import requests
from bs4 import BeautifulSoup
import pandas as pd
import html5lib
import numpy as np
from datetime import datetime
import datefinder


In [2]:

def reprel(text):
    matches = datefinder.find_dates(text)
    for match in matches:
        s=match
        if s:
            return s.date()

    return "In progress"




In [ ]:
df=[]
batch = 64
count = 0
for b in range(batch):
    print("====batch===",b+1)
    for i in range(b*100,(b+1)*100):
        count = count+1
        if count>6368:
            break
        loc ="https://www.malayalachalachithram.com/movie.php?i="+str(i)
        url = requests.get(loc).text
        soup = BeautifulSoup(url,"lxml")
        try:
            table = soup.find("table",{"class":"mdetails"})
            name = soup.select('h1.movie')[0].text.strip()
            t =str(table)
            data = pd.read_html(t)[0]
            df1 = data.T    
            df1.columns = df1.iloc[0]
            df1.drop(df1.index[0],inplace=True)
            df1["Movie"] = name
            cast = []
            print(name)
            divTag = soup.find_all("div", {"class": "heroes"})
            if divTag:
                for tag in divTag:
                    try:
                        cast.append(tag.find("a").text)
                    except:
                        continue
            try:
                tables = soup.findAll("table",{"class":"cast"})
                try:
                    for a in tables:
                        a = table.findAll(lambda tag: tag.name=='a')
                        for c in a:
                            cast.append(c.text)
                except:
                    cast = []
            except:
                continue

            df1["cast"]=[cast]
            df1.Status = df1.Status.apply(reprel)
            df.append(df1)
        except:
            continue
        


In [ ]:
newdf_ =df[0]
for i in range(1,len(df)):
    newdf = newdf.append(df[i])


In [100]:
newdf_.shape

(6267, 25)

In [105]:
newdf.to_pickle("./cinemastat_data.pkl")

In [91]:
## Merging Data

data1 = pd.read_pickle("./dataset/cinemastat_newdf.pkl")
data2 = pd.read_pickle("./dataset/_dataclean.pkl")

In [92]:
data1["cast"] = data1["cast"].apply(set)
column_1 = data1.columns

In [93]:
data1 = data1.reset_index()
data1.head(2)

,index,Art Direction,Background Music,Banner,Choreography,Cinematography,Costumes,Design,Dialogue,Director,...,Music,Original Story,Producer,Screenplay,Shooting Location,Singers,Sound Design,Status,Story,cast
0,1,NaN,NaN,Travancore National Pictures Ltd,NaN,JC Daniel,NaN,NaN,NaN,JC Daniel,...,NaN,NaN,JC Daniel,JC Daniel,NaN,NaN,NaN,1928-07-11,JC Daniel,"{Travancore National Pictures Ltd, Johnson, JC..."
1,1,NaN,NaN,Sree Rajeswari Films,NaN,PE Nayik,NaN,NaN,NaN,PV Rao,...,NaN,NaN,R Sundararaj,PV Rao,NaN,NaN,NaN,1933-12-05,CV Raman Pillai,"{CV Raman Pillai, Sree Rajeswari Films, R Sund..."


In [94]:
data2.rename(columns ={"cast":"leadcast"},inplace = True)

In [95]:
#normalizing column names
columns = [i.lower() for i in data1.columns]
data1.columns = columns

#normalizing column names
columns = [i.lower() for i in data2.columns]
data2.columns = columns

In [96]:
data = pd.merge(data1,data2[['movie','leadcast']],on='movie', how='left')
data.shape

(6374, 27)

In [97]:
#ADDING YEAR to the column derived from the movie name.
import re
def get_year(text):
    pattern = re.compile(r"\((\d+)\)")
    year = ''.join(pattern.findall(text))
    try:
        if int(year)>2020:
            return "UNKNOWN"
        else:
            return int(year)
    except:
        return "UNKNOWN"
        

data["year"] = data.movie.apply(get_year)
data.head(2)

,index,art direction,background music,banner,choreography,cinematography,costumes,design,dialogue,director,...,producer,screenplay,shooting location,singers,sound design,status,story,cast,leadcast,year
0,1,NaN,NaN,Travancore National Pictures Ltd,NaN,JC Daniel,NaN,NaN,NaN,JC Daniel,...,JC Daniel,JC Daniel,NaN,NaN,NaN,1928-07-11,JC Daniel,"{Travancore National Pictures Ltd, Johnson, JC...","[JC Daniel, Johnson, PK Rosy, Sunder Raj]",1928
1,1,NaN,NaN,Sree Rajeswari Films,NaN,PE Nayik,NaN,NaN,NaN,PV Rao,...,R Sundararaj,PV Rao,NaN,NaN,NaN,1933-12-05,CV Raman Pillai,"{CV Raman Pillai, Sree Rajeswari Films, R Sund...","[Pious, Padmini, AP Krishna Menon, Devaki Bhai...",1933


In [98]:
#Rearranging columns
ignore = ["Background Music",'shooting location','original story',"distribution"]
columns =['year','movie', 'movie genre','director', 'editing','producer','banner','screenplay',
          'music','art direction','choreography', 'cinematography', 'costumes', 
          'design', 'dialogue','lyrics', 'makeup',  'singers','story','leadcast','cast','status']
data = data[columns]
data.shape

(6374, 22)

In [99]:
data.tail(5)

,year,movie,movie genre,director,editing,producer,banner,screenplay,music,art direction,...,costumes,design,dialogue,lyrics,makeup,singers,story,leadcast,cast,status
6369,2001,Aarodum Parayathe (2001),NaN,Praveen Dutt,NaN,Gireesh Nair,NaN,Venu Mahadevan,SR Mohan,Shibu Pazhanchira,...,Arafath,NaN,Venu Mahadevan,Bharanikkavu Sivakumar,NaN,"MG Sreekumar, Bhavana Radhakrishnan",Gireesh Nair,"[Sindhu, Adoor Pankajam, Jayanand, Jose Pellis...","{Shibu Pazhanchira, Venu Mahadevan, Bharanikka...",In progress
6370,2002,Aaradyam Parayum (2002),NaN,Paulson,R Sekhar,Minu Kumar,NaN,Babu Pallassery,Aadithyan,Lakshman G Malam,...,Velayudhan Keezhillam,NaN,Babu Pallassery,Bichu Thirumala,Salim Kadakkal(Salim Nagarcoil),"KS Chithra, MG Sreekumar, Biju Narayanan, P Un...",Beena Kumari,"[Gopika, Rakesh]","{Babu Pallassery, Salim Kadakkal(Salim Nagarco...",In progress
6371,1999,Mimics Ghost (1999),Comedy,Paulson,NaN,NaN,NaN,NaN,Johnson,Balan Karumaloor,...,V Ankuswami,NaN,NaN,Mankombu Gopalakrishnan,CV Anil,NaN,NaN,"[Indulekha, Shaju K.S.]","{Indulekha, CV Anil, V Ankuswami, Johnson, Bal...",In progress
6372,1998,Minukkam (1998),NaN,Sasi Mohan,NaN,NaN,NaN,Sidhique Thamarassery,"KV Abutty, Chandrabose",Murali Beppur,...,NaN,NaN,Sidhique Thamarassery,Gireesh Puthenchery,NaN,NaN,NaN,NaN,"{Gireesh Puthenchery, Sasi Mohan, Murali Beppu...",In progress
6373,2011,Ee Abhaya Theeram (2011),NaN,Alex Thankachan,NaN,NaN,NaN,Alex Thankachan,R Somasekharan,NaN,...,NaN,NaN,Alex Thankachan,Bharanikkavu Sivakumar,NaN,"Bhavana Radhakrishnan, Vidhu Prathap, B Shobha...",Alex Thankachan,NaN,"{Shakkeela, R Somasekharan, Bharanikkavu Sivak...",In progress


In [ ]:
## Removing Duplicate names

In [131]:
data["movie"] = data["movie"].str.replace(r"\(.*\)","")
data["movie"] = data["movie"].str.strip()

In [138]:
movie_names = data["movie"].values

In [139]:
import collections
duplicate =[item for item, count in collections.Counter(movie_names).items() if count > 1]

## [1, 2, 5]


In [140]:
duplicate

['Vigathakumaaran', 'Vellinakshathram', 'Sthree', 'Pullimaan', 'Achan', 'Amma', 'Newspaper Boy', 'Thaskaraveeran', 'Chathurangam', 'Seetha', 'Umma', 'Bhaarya', 'Anna', 'Ayisha', 'Sree Guruvayoorappan', 'Inapraavukal', 'Maayaavi', 'Kaliyodam', 'Kuppivala', 'Koottukaar', 'Kayamkulam Kochunni', 'Kanakachilanka', 'Kadamattathachan', 'Thilothama', 'Kunjikkoonan', 'Baalyakaalasakhi', 'Indulekha', 'Kadal', 'Asuravithu', 'Kuruthikkalam', 'Abhayam', 'Kaamuki', 'Devi', 'Lakshyam', 'Prathikaaram', 'Shakthi', 'Upahaaram', 'Chaayam', 'Padmavyooham', 'Kaapaalika', 'Nakhangal', 'Thottaavaadi', 'Yaamini', 'Kaalachakram', 'Udayam', 'Swarna Medal', 'Honeymoon', 'Chattakkaari', 'Suprabhaatham', 'Pattaabhishekam', 'Vrindaavanam', 'Kalyaana Sougandhikam', 'Boy Friend', 'Raasaleela', 'Ayodhya', 'Picnic', 'Sammaanam', 'Criminals', 'Malsaram', 'Sthreedhanam', 'Aalinganam', 'Agnipushpam', 'Romeo', 'Mallanum Mathevanum', 'Chirikkudukka', 'Pushpasharam', 'Thuruppugulaan', 'Lakshmi', 'Agninakshathram', 'Sneham', 

In [145]:
dupdf = data[data.movie.isin(duplicate)]

In [147]:
dupdf.shape

(415, 22)

In [151]:
dupdf[dupdf.movie=='King']

,year,movie,movie genre,director,editing,producer,banner,screenplay,music,art direction,...,costumes,design,dialogue,lyrics,makeup,singers,story,leadcast,cast,status
4150,2009,King,NaN,Srinu Vittala,S Raja,VK Mohammed,Nidhal Pictures,NaN,Devi Sri Prasad,NaN,...,NaN,Sathyans,Satheesh Muthukulam,Siju Thuravoor,NaN,"Afsal, Franco, Rajalakshmi Abhiram, Ranjini Jo...",NaN,"[Nagarjuna, Mamtha Mohandas, Trisha Krishnan]","{Sathyans, Calicut, Nidhal Pictures, Franco, S...",In progress
5325,UNKNOWN,King,NaN,NaN,NaN,NaN,NaN,NaN,Dhina,NaN,...,NaN,NaN,NaN,Mankombu Gopalakrishnan,NaN,NaN,NaN,[],"{Dhina, Mankombu Gopalakrishnan}",In progress


In [45]:
#removing year with unknown release data / All are dubbed movies
data = data[data["year"] != "UNKNOWN" ]
data.shape

(6295, 22)

In [90]:
data = data[~data.movie.isin(duplicate)]

In [44]:
data.shape

(6366, 22)

In [47]:
data.isnull().sum()

year                 0
movie                0
movie genre       6009
director           388
editing           1941
producer          2356
banner            2307
screenplay        1786
music              485
art direction     3225
choreography      6168
cinematography    1701
costumes          5637
design            4087
dialogue          1953
lyrics             705
makeup            5557
singers            955
story             1861
leadcast             3
cast                 0
status               0
dtype: int64

In [77]:
def clean_leadcast(leadcast):
    if isinstance(leadcast, list):
        return int(len(leadcast))
#     else:
#         print(leadcast)
   
                
data["len_cast"] = data.cast.apply(len)
data["len_lead"] = data.leadcast.apply(clean_leadcast)

list(data[["cast","leadcast"]].iloc[1])


[{'CV Raman Pillai', 'Sree Rajeswari Films', 'R Sundararaj', 'A Devaraj', 'PV Rao', 'PE Nayik'}, ['Pious', 'Padmini', 'AP Krishna Menon', 'Devaki Bhai', 'Jayadevan', 'AP Padmanabha Menon', 'R Sundararaj']]

In [85]:
data.head()

,year,movie,movie genre,director,editing,producer,banner,screenplay,music,art direction,...,dialogue,lyrics,makeup,singers,story,leadcast,cast,status,len_lead,len_cast
0,1928,Vigathakumaaran,NaN,JC Daniel,NaN,JC Daniel,Travancore National Pictures Ltd,JC Daniel,NaN,NaN,...,NaN,NaN,NaN,NaN,JC Daniel,"[JC Daniel, Johnson, PK Rosy, Sunder Raj]","{Travancore National Pictures Ltd, Johnson, JC...",1928-07-11,4.0,5
1,1933,Maarthandavarma,NaN,PV Rao,A Devaraj,R Sundararaj,Sree Rajeswari Films,PV Rao,NaN,NaN,...,NaN,NaN,NaN,NaN,CV Raman Pillai,"[Pious, Padmini, AP Krishna Menon, Devaki Bhai...","{CV Raman Pillai, Sree Rajeswari Films, R Sund...",1933-12-05,7.0,6
2,1938,Balan,NaN,S Nottani,PP Varghese,TR Sundaram,Modern Theatres,Muthukulam Raghavan Pilla,"KK Aroor, Ibrahim",NaN,...,Muthukulam Raghavan Pilla,Muthukulam Raghavan Pilla,NaN,"KK Aroor, Master Madanagopal, MK Kamalam, Pall...",A Sundaram Pillai,"[Alleppey Vincent, MK Kamalam, KK Aroor]","{Ibrahim, Sivanandan, KK Aroor, Palluruthi Lak...",1938-01-19,3.0,14
3,1940,Jnaanaambika,NaN,S Nottani,NaN,Annamala Chettiyar,Shyamala Pictures,C Madhavan Pillai,TK Jayarama Iyer,NaN,...,C Madhavan Pillai,Puthankavu Mathan Tharakan,NaN,"CK Rajam, KK Aroor, Mavelikkara L Ponnamma, PK...",C Madhavan Pillai,"[CA Seethalakshmi, Nanukkuttan, Mavelikkara L ...","{CA Seethalakshmi, Puthankavu Mathan Tharakan,...",1940-07-04,10.0,13
4,1941,Prahlaada,Mythology,K Subramaniam,NaN,NaN,Madras United Artist Corporation,NaN,Vidwan VS Parthasarathy Ayyankar,NaN,...,NP Chellappan Nair,K Madhava Warrier,NaN,"Papanasam Sivan, VA Chellappa, Kumari Lakshmi",NaN,"[Guru Gopinath, Kumari Lakshmi, NP Chellappan ...","{Kumari Lakshmi, Madras United Artist Corporat...",1941-08-17,5.0,8


In [81]:
c1 = data.director.isnull()
c2 =data.len_lead == 0
data[c1 & c2]

,year,movie,movie genre,director,editing,producer,banner,screenplay,music,art direction,...,dialogue,lyrics,makeup,singers,story,leadcast,cast,status,len_lead,len_cast
399,1971,Aakashaganga (1971),NaN,NaN,NaN,NaN,NaN,NaN,RK Sekhar,NaN,...,NaN,Sreekumaran Thampi,NaN,"S Janaki, P Leela, Radha",NaN,[],"{Sreekumaran Thampi, Radha, P Leela, RK Sekhar...",In progress,0.0,5
511,1973,Swarna Medal (1973),NaN,NaN,NaN,NaN,NaN,NaN,M Ranga Rao,NaN,...,NaN,"P Bhaskaran, CA Velappan",NaN,"S Janaki, P Susheela",NaN,[],"{P Bhaskaran, M Ranga Rao, P Susheela, CA Vela...",In progress,0.0,5
512,1973,Chithariya Pookkal (1973),NaN,NaN,NaN,NaN,NaN,NaN,"M Ranga Rao, RK Sekhar",NaN,...,NaN,"CA Velappan, Gandhari",NaN,"KJ Yesudas, S Janaki, Sobha",NaN,[],"{Gandhari, KJ Yesudas, Sobha, M Ranga Rao, RK ...",In progress,0.0,7
561,1974,Vasantharaavukal (1974),NaN,NaN,NaN,NaN,NaN,NaN,Kottayam Joy,NaN,...,NaN,Bharanikkavu Sivakumar,NaN,"KJ Yesudas, P Jayachandran, P Madhuri, Ambili",NaN,[],"{P Jayachandran, KJ Yesudas, Bharanikkavu Siva...",In progress,0.0,6
635,1975,Ninne Pinne Kandolaam (1975),NaN,NaN,NaN,NaN,NaN,NaN,Sathyam,NaN,...,PG Vasudevan,Poochakkal Shahul Hameed,NaN,"S Janaki, P Jayachandran",NaN,[],"{P Jayachandran, PG Vasudevan, Poochakkal Shah...",1975-04-14,0.0,5
801,1977,Kaamalola (1977),NaN,NaN,NaN,NaN,NaN,NaN,Cochin Alex,NaN,...,NaN,"Melady Ravi, RK Ravivarma",NaN,"S Janaki, Jolly Abraham, Latha Raju, Selma Geo...",NaN,[],"{Cochin Alex, Sherly, Jolly Abraham, RK Raviva...",In progress,0.0,8
802,1977,Cheruppakkar Sookshikkuka (1977),NaN,NaN,NaN,NaN,NaN,NaN,V Dakshinamoorthy,NaN,...,NaN,"Vayalar, Kallayam Krishnadas",NaN,"KJ Yesudas, P Susheela, Ambili",NaN,[],"{Vayalar, Kallayam Krishnadas, V Dakshinamoort...",In progress,0.0,6
931,1978,Kadalkkaakkakal (1978),NaN,NaN,NaN,NaN,NaN,NaN,AT Ummer,NaN,...,NaN,Poovachal Khader,NaN,"KJ Yesudas, S Janaki, Vani Jairam",NaN,[],"{KJ Yesudas, Poovachal Khader, Vani Jairam, S ...",In progress,0.0,5
932,1978,Thurakkoo Oru Vaathil (1978),NaN,NaN,NaN,NaN,NaN,NaN,K Raghavan,NaN,...,NaN,VT Kumaran,NaN,"S Janaki, Selma George",NaN,[],"{K Raghavan, VT Kumaran, S Janaki, Selma George}",In progress,0.0,4
936,1978,Kshethram (1978),NaN,NaN,NaN,NaN,Kalabharathi Films,NaN,Kannur Rajan,NaN,...,NaN,Bharanikkavu Sivakumar,NaN,"KJ Yesudas, Ambili, KP Brahmanandan",NaN,[],"{KJ Yesudas, KP Brahmanandan, Bharanikkavu Siv...",In progress,0.0,6
